# Projet Réseaux euclidiens en cryptographie
###  [@cypri3](https://github.com/cypri3/)

## Première partie : Implémentation du système cryptographique

In [1]:
"""
Génère une matrice unimodulaire aléatoire.

Paramètres :
- n (int) : Taille de la matrice carrée (n x n).
- nbc (int) : Nombre de matrices unimodulaires multipliées entre elles (valeur suggérée : 2n).

Retour :
- Matrix : Une matrice unimodulaire de dimensions n x n (déterminant égal à ±1).
"""
def generate_unimodular_matrix(n, nbc = 1):
    Ures = matrix.identity(n)
    dist = GeneralDiscreteDistribution([1/7, 5/7, 1/7])
    
    for i in range(nbc):
        U = matrix.identity(n)
        ind = randint(0,n-1)
        
        if randint(0,1): # Modifie une ligne ou une colonne avec 50% de chances
            U[ind] = [dist.get_random_element() - 1 for _ in range(n)]
            U[ind, ind] = 1
        else:
            for i in range(n):
                U[i, ind] = dist.get_random_element() - 1
            U[ind, ind] = 1
        Ures *= U
    return Ures

In [2]:
"""
Génère une clé publique et une clé privée pour un chiffrement basé sur des matrices.

Paramètres :
- n (int) : Taille des matrices carrées (n x n).
- l (int) : Amplitude des coefficients dans la matrice privée (valeur par défaut : 4).
- debug (bool) : Si True, affiche les matrices générées pour le débogage.
- nbc (int) : Nombre de matrices unimodulaires multipliées entre elles.

Retour :
- Bpriv (Matrix) : Matrice privée.
- Bpub (Matrix) : Matrice publique.
- U (Matrix) : Matrice unimodulaire utilisée pour transformer Bpriv en Bpub.
"""
def KeyGen(n, l = 4, debug = False, nbc = 1):
    k = round(sqrt(n)) * 4
    IDk = k * matrix.identity(n)
    Bpriv = IDk + MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    while Bpriv.det() == 0:
        Bpriv = IDk + MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    Bpriv = Bpriv.LLL()
        
    U = generate_unimodular_matrix(n, nbc=nbc)
        
    Bpub = U * Bpriv
    
    if(debug):
        print("Matrice B (clé privée) :")
        print(Bpriv)
        print("\nMatrice U (unimodulaire) :")
        print(U)
        print("\nMatrice B' (clé publique) :")
        print(Bpub)

    return Bpriv, Bpub, U

In [3]:
"""
Chiffre un message à l'aide de la clé publique.

Paramètres :
- Bpub (Matrix) : Clé publique utilisée pour le chiffrement.
- m (vector) : Message à chiffrer, sous la forme d'un vecteur.
- breakPoint (int) : Nombre maximal d'itérations pour ajuster l'erreur (valeur par défaut : 128).

Retour :
- c (vector) : Message chiffré
"""
def Cipher(Bpub, m, breakPoint = 128):
    n = Bpub.nrows()
    r = round(min([Bpub.column(i).norm(2) for i in range(n)]) / 6)
    err = vector(ZZ, [randint(-1, 1) for _ in range(n)])
    boucle = 0
    while err.norm(2) >= r:
        boucle += 1
        err = vector(ZZ, [randint(-1, 1) for _ in range(n)])
        if(boucle > breakPoint):
            err = vector(ZZ, [randint(0,1) if(i < r) else 0 for i in range(n)])
            break
    return m*Bpub + err

"""
Déchiffre un message à l'aide de la clé privée.

Paramètres :
- Bpriv (Matrix) : Matrice privée pour le déchiffrement.
- U (Matrix) : Matrice unimodulaire utilisée dans la génération des clés.
- c (vector) : Message chiffré.

Retour :
- m (vector) : Message clair, sous la forme d'un vecteur.
"""
def Decipher(Bpriv, U, c):
    m_prim = c * Bpriv.inverse()
    m_sec = m_prim.apply_map(lambda x: round(x))
    
    # Autre version qui peut être préférable sur la version consôle de sage
    # m_sec = m_prim.apply_map(lambda x: math.ceil(x) if abs(x % 1) == 0.5 else round(x))
    
    m = m_sec * U.inverse()

    return m

In [4]:
"""
Teste les fonctions de génération de clé, de chiffrement et de déchiffrement.

Paramètres :
- n (int) : Taille des matrices carrées utilisées pour le chiffrement.
- nb (int) : Nombre de tests à effectuer.
"""
def test_encryption_system(n, nb):
    good = 0
    ERR = []
    nberr = 0

    for i in range(nb):
        Bpriv, Bpub, U = KeyGen(n, nbc = 2 * n)

        m = vector(ZZ, [randint(0, 1) for _ in range(n)])

        c = Cipher(Bpub, m)

        m2 = Decipher(Bpriv, U, c)

        if m == m2:
            good += 1
        else:
            nberr += 1
            e = m - m2
            print(f"Erreur détectée : {e}")
            ERR.append(sum(1 for j in e if j != 0))

    taux_reussite = (good / nb) * 100
    taux_erreur_bits = (sum(ERR) / (nberr * n)) * 100 if nberr > 0 else 0

    print(f"Le taux de réussite dans le déchiffrement pour des matrices de taille {n} est de {round(taux_reussite)}%")
    if nberr > 0:
        print(f"Le taux de bits incorrectement déchiffrés en cas d'erreur est de {round(taux_erreur_bits)}%")
        
n, nb = 20, 10
test_encryption_system(n, nb)

Le taux de réussite dans le déchiffrement pour des matrices de taille 20 est de 100%


In [5]:
"""
Chiffre une chaîne de caractères en utilisant une clé publique.

Paramètres :
- message (str) : Chaîne à chiffrer.
- Bpub (Matrix) : Matrice publique générée.
- size (int) : Taille des blocs pour le chiffrement (par défaut : 112).

Retour :
- list : Liste des vecteurs chiffrés.
"""
def encrypt_string(message, Bpub, size=112):
    # Convertion de chaque caractère en 8 bits
    m_bin = ''.join([bin(ord(char))[2:].zfill(8) for char in message])

    # Ajouter du padding
    n_max = len(m_bin)
    r = n_max % size
    m_bin += "0" * (size - r) if r != 0 else ""

    # Découpe en vecteurs de taille fixe
    list_vec = [vector([int(bit) for bit in m_bin[i * size:(i + 1) * size]]) 
                for i in range(len(m_bin) // size)]

    C = [Cipher(Bpub, m) for m in list_vec]
    return C

"""
Déchiffre une liste de vecteurs chiffrés.

Paramètres :
- C (list) : Liste des vecteurs chiffrés.
- Bpriv (Matrix) : Matrice privée générée.
- U (Matrix) : Matrice unimodulaire utilisée pour la clé publique.
- size (int) : Taille des blocs pour le chiffrement (par défaut : 112).

Retour :
- str : Chaîne déchiffrée.
"""
def decrypt_string(C, Bpriv, U, size=112):
    M2 = [Decipher(Bpriv, U, c).list() for c in C]

    # Retrait du padding
    while M2[-1][-8:] == [0] * 8:
        M2[-1] = M2[-1][:-8]

    # Reconstitution de la chaîne binaire
    M2 = ''.join([''.join(map(str, block)) for block in M2])
    message = ''.join([chr(int(M2[i:i+8], 2)) for i in range(0, len(M2), 8)])
    return message

"""
Teste le chiffrement et le déchiffrement d'une chaîne de caractères.

Paramètres :
- message (str) : Chaîne à chiffrer et déchiffrer.
- size (int) : Taille des blocs pour le chiffrement (par défaut : 112).
- debug (bool) : Si True, affiche les matrices générées pour le débogage.
"""
def test_global_system(message, size = 8 * 14, debug = False, Bpriv= None, Bpub = None, U = None):
    print("Message d'origine :", message)
    
    if (Bpriv == None or Bpub == None or U == None) or (Bpriv.ncols() % 8 != 0):
        if(size % 8 != 0):
            size = 8 * 14
        Bpriv, Bpub, U = KeyGen(size, debug=debug)

    encrypted_message = encrypt_string(message, Bpub, size)
    print("\nMessage chiffré :", encrypted_message)

    decrypted_message = decrypt_string(encrypted_message, Bpriv, U, size)
    print("\nMessage déchiffré :", decrypted_message)

    if message == decrypted_message:
        print("\nLe message a été correctement déchiffré.")
    else:
        print("\nErreur dans le déchiffrement.")

test_global_system("Ceci est un test de chiffrement et déchiffrement")

Message d'origine : Ceci est un test de chiffrement et déchiffrement

Message chiffré : [(22, 5, 41, 68, 9, -13, 70, 69, -19, -3, 3, 51, -8, -6, -33, 42, 49, 32, 64, 30, 76, 60, 7, 6, 44, -2, 35, 71, 21, -14, 60, 69, 30, 9, 9, 6, 26, -14, -25, 55, 24, 5, 2, 45, -4, 79, -18, 58, -6, 12, 9, 25, 6, -21, 5, 55, 46, 12, 20, 18, 54, -6, 3, 75, 4, -8, 31, 61, 4, -3, 20, 8, 59, -30, 7, 32, 49, 31, 15, -2, -12, 5, -7, 22, 26, 35, 42, 55, 26, 22, 5, 7, 2, 42, -5, 58, 10, 39, 67, -20, 30, 3, 55, 9, 39, -9, 7, -3, 4, -3, 30, 21), (64, 9, 33, 45, 18, -11, 59, 52, -5, -1, 8, 37, 42, -20, -12, 65, 42, -12, 58, 34, 68, 36, 34, 37, 39, 1, 23, 63, -17, -27, 10, 59, 62, 17, 21, 12, 23, -21, 23, -14, 37, 10, 8, -11, 7, 71, -1, 41, 36, 55, -10, 20, 3, -23, -11, 31, -33, -3, 37, 64, 58, -16, -4, 31, 6, 41, -1, 60, 7, -3, 37, 18, 9, 12, 10, 52, 66, -9, 13, -13, 42, 10, 43, 18, 20, 50, -1, 57, 52, 47, -8, 34, -10, 14, -42, 65, -19, 36, 13, -9, 84, 55, 89, 11, 16, -8, 31, 33, 2, 1, 39, 13), (109, 6, 34, 58, 14

## Cryptanalyse 

### Attaque par force brute

In [6]:
from itertools import product

"""
Génère tous les vecteurs possibles de dimension donnée avec des coefficients
dans {0, -1, 1} et une norme inférieure à une borne.

Paramètres :
- n (int) : Taille des vecteurs à générer.
- r (int) : Borne supérieure pour la norme Euclidienne.

Retour :
- (iterator) : Itérateur sur les vecteurs respectant les contraintes.
"""
def generate_vectors_iterative(n, r):
    for v in product([0, -1, 1], repeat=n):
        v = vector(v)
        if v.norm(2) < r:
            yield v

"""
Effectue une attaque par force brute pour déchiffrer un message.

Paramètres :
- Bpub (Matrix) : Matrice publique utilisée pour le chiffrement.
- c (vector) : Message chiffré.
- borneInf (int) : Borne inférieure des coefficients des vecteurs candidats (par défaut : 0).
- borneSup (int) : Borne supérieure exclusive des coefficients des vecteurs candidats (par défaut : 2).
  C'est-à-dire que l'on définit l'espace des coefficients du vecteur m comme [borneInf, borneSup[

Retour :
- candidates (list) : Liste des messages candidats sous forme de vecteurs.

Note : Les bornes définissent l'espace de recherche des messages candidats. Avec les valeurs 
par défaut, la fonction cherche un message binaire.
"""

def brute_force_attack(Bpub, c, m, borneInf=0, borneSup=2):
    n = Bpub.ncols()
    r = round(min([Bpub.column(i).norm(2) for i in range(n)]) / 6)
    Bpub_inv = Bpub.inverse()
    candidates = []
    
    c_prim = c * Bpub_inv
    for e in generate_vectors_iterative(n, r):
        possible_m = c_prim - Bpub_inv * e
        possible_m = possible_m.apply_map(lambda x: round(x))

        if all(x in range(borneInf,borneSup) for x in possible_m):
            candidates.append(possible_m)

            if possible_m == m:
                print("Message récupéré :", possible_m)
                return candidates

    print("Message non trouvé, augmentation de la norme maximale nécessaire.")
    return candidates

In [7]:
n = 5
Bpriv, Bpub, U = KeyGen(n, nbc=1)

m = vector(ZZ, [randint(0, 1) for _ in range(n)])
print("Message recherché :",m)
c = Cipher(Bpub, m)
candidats = brute_force_attack(Bpub, c, m)
if len(candidats) > 1:
    print("Liste des premiers candidats :", candidats)

Message recherché : (1, 1, 1, 1, 0)
Message récupéré : (1, 1, 1, 1, 0)


In [8]:
"""
Effectue une attaque par l'algorithme Nearest-Plane sur un message chiffré.

Paramètres :
- c (vector) : Le message chiffré.
- Bpub (Matrix) : La matrice publique utilisée pour le chiffrement.

Retour :
- e_new (vector) : Une approximation de l'erreur.
"""
def nearest_plane_error(c, Bpub):
    n = Bpub.nrows()
    Bpub_lll = Bpub.LLL()
    
    Bpub_inv = Bpub_lll.inverse()

    c_prim = c * Bpub_inv
    
    coefs = [round(i) for i in c_prim.list()]

    Bnew = Bpub_lll
    e_new = c

    for i in range(n-1, -1, -1):
        e_new = e_new - coefs[i] * Bnew[i]
        Bnew = matrix(list(Bnew)[:i])
        
    return e_new

In [9]:
"""
Effectue une attaque complète pour déchiffrer un message à l'aide de l'algorithme Nearest-Plane.

Paramètres :
- c (vector) : Message chiffré.
- Bpub (Matrix) : Matrice publique utilisée pour le chiffrement.

Retour :
- m (vector) : Message clair approximé.
"""
def nearest_plane_attack(Bpub, c):
    error = nearest_plane_error(c, Bpub)
    
    Bpub_inv = Bpub.inverse()
    
    m_prim = c * Bpub_inv - Bpub_inv * error
    m = m_prim.apply_map(lambda x: round(x))
    
    return m

In [10]:
n = 200
Bpriv, Bpub, U = KeyGen(n)
m = vector([randint(-n^2,n^2) for i in range(n)])

c = Cipher(Bpub, m)

m_recovered = nearest_plane_attack(Bpub, c)

print("Message recherché :", m)
print()
print("Message récupéré :", m_recovered)
print()
if m == m_recovered:
    print("Le message à été correctement retrouvé.")
else:
    print("Le message retrouvé ne correspond pas au message original.")

Message recherché : (18675, 13903, -35657, 28940, 10237, 14536, -470, 21468, -32065, -9357, 38477, -9194, -17928, 11352, 37382, -33398, -20932, -12188, -38947, 24138, 18866, -13680, -26315, 19506, -3730, 24376, 14189, -34711, 8680, 36062, -2820, -29883, 27115, -37946, -16592, 17171, -11863, -1736, -626, -37511, 38782, 8332, -36174, -17969, -15519, -35050, -21999, -15949, 23849, -6725, -1298, -8589, 5484, 39639, -23619, -22099, -10358, 28493, -8347, -24, -35368, 25535, -9087, 12729, 24448, -7153, -34290, -26420, -32367, -18674, 35186, -37466, -19797, -32992, 7727, 27492, 25948, 11855, -25499, 30560, 23325, 13803, 16845, -16865, -8953, 400, -18449, 30177, -15229, -36180, -2432, -5298, -38695, 16306, 35878, 34118, 15522, 6877, -29567, -22585, -38495, -26098, -1161, -16183, -29735, -5096, 29252, 26853, 26865, 35713, 33885, -23080, 13764, 20674, -23450, 6674, 26933, 26494, -17287, 19503, 27297, -11177, 34209, -26472, -29480, 21734, 37454, 24836, 21766, -38826, -10998, 35650, -28811, -36861,

In [11]:
""" 
Effectue une attaque d'embedding pour déchiffrer un message à l'aide de la réduction LLL.

Paramètres :
- c (vector) : Message chiffré.
- Bpub (Matrix) : Matrice publique utilisée pour le chiffrement.

Retour :
-m (vector) : Message clair approximé.
"""
def embedding_attack(Bpub, c):
    n = Bpub.nrows()
    
    B = matrix(Bpub.rows() + [c]).transpose()
    B = matrix(B.rows() + [[ i == n for i in range(n+1)]]).transpose().LLL()
    
    e = vector(B[0][:n])
    
    m = (c - e) * Bpub.inverse()
    
    
    return m.apply_map(round)

In [12]:
n = 200
Bpriv, Bpub, U = KeyGen(n)
m = vector([randint(-n^2,n^2) for i in range(n)])

c = Cipher(Bpub, m)

m_recovered = embedding_attack(Bpub, c)

print("Message recherché :", m)
print()
print("Message récupéré :", m_recovered)
print()
if m == m_recovered:
    print("Le message à été correctement retrouvé.")
else:
    print("Le message retrouvé ne correspond pas au message original.")

Message recherché : (2148, -21036, 27632, 13122, -37758, 27173, 26625, 28736, 7148, -27946, -23465, -38146, -36028, -17004, -5926, 23075, -34071, -17081, -26675, -23790, 26817, -20261, 27536, 6998, -32008, -16841, 25835, 115, 23512, 4098, 16251, 14499, 28388, -32897, 20620, -34044, 31153, -26040, -11765, 34146, 15009, -14333, 3604, -35192, -29775, 29973, 18508, 34713, -19445, 8433, 12816, 33902, 23812, 15139, 1682, -31103, -33039, -39760, -11178, 31990, -14129, 17084, 24188, 10147, 21108, 1471, -18153, 702, 29700, -32400, -22237, -20449, -1889, 34052, -18425, 16269, 23871, -11928, -19612, 35750, -7817, -37094, 37946, -1952, 25850, 27716, -27724, -25919, 7686, -16539, 20751, -18219, -8350, -18462, -3882, -6269, -3547, -25629, 16653, -32155, 20232, -8074, 25091, 5850, -37303, -7982, 26910, 22104, 20399, -12243, -11270, 2802, -34119, 5458, 6292, 21447, 7319, -28823, 36275, 29110, 20605, -19036, 24509, 7650, -35985, 2054, 16862, 24811, 6920, 32522, -19790, -31143, -20150, -13847, -18303, 1

## Partie tests généraux

In [13]:
def test(n = 16):
    Bpriv, Bpub, U = KeyGen(n, debug = True, nbc=2*n)
    message = "Ceci est un test de chiffrement et déchiffrement"
    print("\nVérification du système de chiffrement :\n")
    test_global_system(message, size = n, debug = False, Bpriv = Bpriv, Bpub = Bpub, U = U)

    print("\nRéalisation des attaques :\n")
    Bpriv, Bpub, U = KeyGen(n, nbc=4) # nbc faible pour des raisons de performances
    
    print("1. Attaque par force brute")
    
    m = vector(ZZ, [randint(-5, 5) for _ in range(n)])
    print("Message recherché :",m)
    c = Cipher(Bpub, m)
    candidats = brute_force_attack(Bpub, c, m, borneInf = -5, borneSup = 6)
    if len(candidats) > 1:
        print("Liste des premiers candidats :", candidats)
        
    print("\n2. Attaque nearest plane :\n")
    
    m_recovered = nearest_plane_attack(Bpub, c)
    print("Message recherché :", m)
    print("Message récupéré :", m_recovered)
    if m == m_recovered:
        print("Le message à été correctement retrouvé.")
    else:
        print("Le message retrouvé ne correspond pas au message original.")
        
    print("\n3. Attaque embedding :\n")
    
    m_recovered = embedding_attack(Bpub, c)
    print("Message recherché :", m)
    print("Message récupéré :", m_recovered)
    if m == m_recovered:
        print("Le message à été correctement retrouvé.")
    else:
        print("Le message retrouvé ne correspond pas au message original.")
test()

Matrice B (clé privée) :
[ 0 -2  3 -3  3 12 -1  3  1 -4  4 -3  2  1  3  1]
[-3  5  1 -1 -6 -9 -3  0  1  1 10  3 -1  1 -1  1]
[-1  1 -2 -2  0  3 13  3  4  1  3 -3 -1 -4 -3  3]
[ 4 -2 -3  4 -2 -1  1 -2 13  4 -3  4  4  2 -3 -4]
[15  0  2 -3 -4 -2  0 -4 -2  3 -2 -4  0 -3 -1  2]
[ 7 12  3 -3  8  7  2  2 -3 -3 -6 -6 -3 -5  4  3]
[ 4 -3  3 14 -4  0 -2  1 -1 -4 -1 -3 -4 -4  2  2]
[ 3  1 -2 -1 18 -4 -2 -3  3  1 -4  1  4 -2  1  4]
[-1 -3 -1 -3  0 -1  3  1 -3  4 -4  0 -3 19 -1 -2]
[-1  0 -2 -1  2  4  3 -3  3 -4 -1 17  0 -3 -4  2]
[ 4  0 -4 -2  0  1 -2  1 -3 -3  4  3  0  3 17  4]
[-2  1 -1 -1  1 14  4  2  4 12  2 12 -2 -6  3 -1]
[-1  3  0  1  2  4  4  0  0 -4  2 -2 19  0  1  3]
[ 1  4 -2 -4  3  1 -2 -4 -4  3 -1 -1  4  3  0 19]
[-1 -1 18  3 -1 -3  4 -1 -1  4 -2  1  3  1  2  1]
[ 6 -1 -3 -4 -3  8 -8 12  2  4 -5  7 13 -1  2  4]

Matrice U (unimodulaire) :
[  0   0   1   1  -3  -1   0   2  -1   0   1   1   1   0   8   0]
[  0   3  -5  -2   7  -2  -2  -5   4   2  -6  -8  -2   0 -25   5]
[ -3   3 -10 -1